## 在做seq2seq的时候，经常需要使用sequence_loss这是损失函数。


####  <font color=#FF0000 >原创文章，转载请注明出处 </font>
   
现在分析一下sequence_loss这个函数到底在做什么

```
# coding: utf-8
import numpy as np
import tensorflow as tf
from tensorflow.contrib.seq2seq import sequence_loss

logits_np = np.array([
   [[1.0, 2.0], [1.0, 2.0]],
    [[1.0, 2.0], [1.0, 2.0]]
])

targets_np = np.array([
    [0,1],
    [1,1]
], dtype=np.int32)

logits = tf.convert_to_tensor(logits_np)
targets = tf.convert_to_tensor(targets_np)
cost = sequence_loss(logits=logits,
                     targets=targets,
                     weights=tf.ones_like(targets, dtype=tf.float64))
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    r = sess.run(cost)
    print(r)
    
# sequence_loss的结果是0.563261687518082

```

求loss值
$$
logits=\left[\begin{matrix} [1.0, 2.0] & [1.0, 2.0] \cr [1.0, 2.0] & [1.0, 2.0]\end{matrix}\right]
$$
$$
target=\left[\begin{matrix} 0.0 & 1.0 \cr 1.0 & 1.0 \end{matrix}\right]
$$
$$
cost=sequence\_loss( logits=logits,targets=targets,weights=tf.ones_like(targets, dtype=tf.float64))
$$


### sequence_loss的求值过程
1.softmax求值 

2.交叉熵选择

3.求平均值



### 1.softmax

将得分或者概率fi，统一转化到0到1之间，就是计算权重占比（归一化处理）
但是在计算权重的时候，分数都通过自然数e映射转换，<font color=#FF0000 >目的是，让大的分数更大，让小的分数更小，增加区分度</font>
$$f_i(z)=-log( \frac{ e^{f_i} }{ \sum{e^{f_j} }} )$$

其输入值是一个向量，向量中元素为任意实数的得分值

输出一个向量，其中每个元素值在0到1之间，且所有元素之和为1（计算每个得分在总分中的占比。<font color=#FF0000 >这里通过指数映射了一下 </font>）


$$f_i(z)=-log( \frac{ e^{f_i} }{ \sum{e^{f_j} }} )$$


logits = [
    [[1.0, 2.0], [1.0, 2.0]],
    [[1.0, 2.0], [1.0, 2.0]]
]

$$
softmax=\left[\begin{matrix} [ \frac{ e^{1.0} }{ e^{1.0}+e^{2.0}}  ,  \frac{ e^{2.0} }{ e^{1.0}+e^{2.0}}] & [\frac{ e^{1.0} }{ e^{1.0}+e^{2.0}},\frac{ e^{2.0} }{ e^{1.0}+e^{2.0}}] \cr [\frac{ e^{1.0} }{ e^{1.0}+e^{2.0}},\frac{ e^{2.0} }{ e^{1.0}+e^{2.0}}] & [\frac{ e^{1.0} }{ e^{1.0}+e^{2.0}},\frac{ e^{2.0} }{ e^{1.0}+e^{2.0}}]\end{matrix}\right]
$$


### 2求交叉熵


targets = [
    [0,1],
    [1,1]
    ]
根据targets, 确定选取哪个值。


$$
crross\_softmax=\left[\begin{matrix} -log(\frac{ e^{1.0} }{ e^{1.0}+e^{2.0}}) & -log(\frac{ e^{2.0} }{ e^{1.0}+e^{2.0}}) \cr -log(\frac{ e^{2.0} }{ e^{1.0}+e^{2.0}}) & -log(\frac{ e^{2.0} }{ e^{1.0}+e^{2.0}}\end{matrix}) \right]
$$


```
再求平均值
loss1=(-log(2.718/(2.718+7.387))+(-log(7.387/(2.718+7.387))))/2
loss2=(-log(7.387/(2.718+7.387))+(-log(7.387/(2.718+7.387))))/2
loss=(loss1+loss2)/2
loss=0.563
```